# Import Lib

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import shutil
from tensorflow.keras.callbacks import CSVLogger


# Import Dataset

In [ ]:
# !!! กรุณาแก้ไข datapath ให้เป็น path ที่ถูกต้องไปยังชุดข้อมูลของคุณ !!!
train_path = '/media/capybara/Data/dataset_vit/archive'
datapath = train_path

# Preprocessing Image

In [ ]:
# --- กำหนดค่าพารามิเตอร์เริ่มต้น ---
img_height = 224
img_width = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

# --- 1. โหลดชุดข้อมูลโดยใช้ image_dataset_from_directory ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    datapath,
    validation_split=0.1,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True, 
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    datapath,
    validation_split=0.1,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

class_names = train_dataset.class_names
num_classes = len(class_names)
print("\n--- ตรวจสอบข้อมูลชุดข้อมูล ---")
print("Number of classes:", num_classes)
print("Class names:", class_names)

# --- 2. สร้าง Model สำหรับ Data Augmentation และ ResNet50 Preprocessing ---
# Import ฟังก์ชัน preprocess_input ของ ResNet50
data_augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(factor=(-10/360, 10/360), fill_mode='nearest'),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest'),
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), fill_mode='nearest')
])

# --- 3. สร้างฟังก์ชันสำหรับนำ Augmentation และ Preprocessing ไปใช้ ---
def augment_and_preprocess_train_data(images, labels):
    images = data_augmentation_layers(images, training=True)
    images = resnet_preprocess_input(images)
    return images, labels

def preprocess_val_data(images, labels):
    images = resnet_preprocess_input(images)
    return images, labels

# --- 4. สร้าง Input Pipelines ที่มีประสิทธิภาพ ---
train_pipeline = train_dataset.map(augment_and_preprocess_train_data, num_parallel_calls=AUTOTUNE)
train_pipeline = train_pipeline.prefetch(buffer_size=AUTOTUNE)

val_pipeline = val_dataset.map(preprocess_val_data, num_parallel_calls=AUTOTUNE)
val_pipeline = val_pipeline.prefetch(buffer_size=AUTOTUNE)

# # --- ตรวจสอบ Output ของ Datasets ---
# print("\n--- ตรวจสอบ Output ของ Datasets (หลัง Preprocessing) ---")
# for X_batch_train, y_batch_train in train_pipeline.take(1):
#     print("Shape of first BATCH of TRAIN images:", X_batch_train.shape)
#     print("Data type of TRAIN images:", X_batch_train.dtype)
#     print("Min value in TRAIN images:", tf.reduce_min(X_batch_train).numpy())
#     print("Max value in TRAIN images:", tf.reduce_max(X_batch_train).numpy())
#     print("Shape of first BATCH of TRAIN labels:", y_batch_train.shape)

# for X_batch_val, y_batch_val in val_pipeline.take(1):
#     print("Shape of first BATCH of VAL images:", X_batch_val.shape)
#     print("Data type of VAL images:", X_batch_val.dtype)
#     print("Min value in VAL images:", tf.reduce_min(X_batch_val).numpy())
#     print("Max value in VAL images:", tf.reduce_max(X_batch_val).numpy())
#     print("Shape of first BATCH of VAL labels:", y_batch_val.shape)

# Load the Pre-trained ResNet50 (Feature Extractor) and retrain only the Head.

## Phase 1

In [ ]:
# --- กำหนดค่าพื้นฐาน ---
IMAGE_SIZE = (224, 224)
INITIAL_LR = 1e-3
FINE_TUNE_LR = 1e-5

# --- ขั้นตอนที่ 1: โหลด Pre-trained ResNet50 (Feature Extractor) และฝึกสอนเฉพาะ Head ใหม่ ---

# === โหลด ResNet50 Pre-trained บน ImageNet, ไม่รวม Head เดิม ===
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + (3,))

# === Freeze Backbone Layers ===
base_model.trainable = False # Freeze layers ของ ResNet50 เดิมทั้งหมด

# === สร้าง Head ใหม่ ===
# ต่อ GlobalAveragePooling2D เพื่อลดมิติ ต่อด้วย Dense layer สำหรับ classification
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dense(1024, activation='relu', name='fc1')(x) # อาจจะมี Dense layer ขั้นกลาง
x = Dropout(0.5, name='dropout')(x) # เพิ่ม Dropout เพื่อป้องกัน Overfitting
predictions = Dense(num_classes, activation='softmax', name='custom_classifier')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.summary() # ดูโครงสร้างโมเดล

# === Compile โมเดลสำหรับ Phase 1 ===
model.compile(
    optimizer=Adam(learning_rate=INITIAL_LR),
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5, # จำนวน epoch ที่จะรอถ้า val_loss ไม่ดีขึ้น
    restore_best_weights=True
)

# สร้าง ModelCheckpoint callback
model_checkpoint_path = 'best_resnet_model_imagenet2012v2_classes.keras' # หรือ .h5 หรือ tf format
model_checkpoint = ModelCheckpoint(
    filepath=model_checkpoint_path, 
    monitor='val_loss',      # เกณฑ์ที่ใช้ในการเลือกโมเดลที่ดีที่สุด
    save_best_only=True,     # บันทึกเฉพาะโมเดลที่ดีที่สุด
    save_weights_only=False, # บันทึกทั้งสถาปัตยกรรมและน้ำหนัก (ถ้า False) หรือเฉพาะน้ำหนัก (ถ้า True)
    verbose=1                # แสดงข้อความเมื่อมีการบันทึก
)

csv_logger = CSVLogger('training_log.csv', append=True)


callback_list_phase1 = [early_stopping, model_checkpoint, csv_logger]
print("--- เริ่ม Phase 1: Training the new head (ResNet50) ---")
# --- Training Phase 1 ---
num_epochs_phase1 = 10 # กำหนดจำนวน epochs
history_phase1 = model.fit(
    train_pipeline, 
    epochs=num_epochs_phase1,
    validation_data=val_pipeline, 
    callbacks=callback_list_phase1
)
print("--- จบ Phase 1 ---")

## Phase 2 Fine tune

In [ ]:
# --- ขั้นตอนที่ 2: Unfreeze Backbone (หรือบางส่วน) และ Fine-tune ทั้งโมเดล ---
print("\n Loading the best model from Phase 1 for fine-tuning...")
model.load_weights(model_checkpoint_path) # โหลดโมเดลที่ดีที่สุดจาก Phase 1
# === Unfreeze Backbone Layers ===
base_model.trainable = True

fine_tune_at = 143 # Index ของ layer 'conv5_block1_0_conv' ใน ResNet50 (อาจจะต้องเช็คชื่อ layer อีกครั้ง)
print(f"Fine-tuning from layer index {fine_tune_at} ('{base_model.layers[fine_tune_at].name}') onwards.")
for layer in base_model.layers[:fine_tune_at]:
   layer.trainable = False

# === Re-compile โมเดลสำหรับ Phase 2 ด้วย learning rate ที่ต่ำกว่า ===
model.compile(
    optimizer=Adam(learning_rate=FINE_TUNE_LR), # Learning rate ต่ำมาก
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary() # ดูว่าพารามิเตอร์ที่ trainable เพิ่มขึ้น

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # ลด learning rate ลง 10 เท่า
    patience=2, # จำนวน epoch ที่จะรอถ้า val_loss ไม่ดีขึ้น
    min_lr=1e-7, # ค่าต่ำสุดของ learning rate
    verbose=1
)

# log_dir_phase2 = "logs/fit/phase2_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback_phase2 = TensorBoard(log_dir=log_dir_phase2, histogram_freq=1)

callback_list_phase2 = [early_stopping, model_checkpoint, reduce_lr, csv_logger]

print("\n--- เริ่ม Phase 2: Fine-tuning the entire model (ResNet50) ---")
# --- Training Phase 2 ---
num_epochs_phase2 = 20 # กำหนดจำนวน epochs
# # ถ้า train ต่อจาก phase 1:
if history_phase1.epoch:
    initial_epoch_phase2 = history_phase1.epoch[-1] + 1 # เริ่มจาก epoch ถัดไป
else:
    initial_epoch_phase2 = 0

total_epochs_overall = initial_epoch_phase2 + num_epochs_phase2

history_phase2 = model.fit(
    train_pipeline,
    epochs=total_epochs_overall, 
    initial_epoch=initial_epoch_phase2, # ถ้า train ต่อ
    validation_data=val_pipeline,
    callbacks=callback_list_phase2
)
print("--- จบ Phase 2 ---")

# Evaluate the model for accuracy

In [ ]:
print("\n--- Evaluating the BEST model saved by ModelCheckpoint ---")
# โหลดโมเดลที่ดีที่สุดที่บันทึกโดย ModelCheckpoint
# best_model_path ควรตรงกับ filepath ใน ModelCheckpoint callback
best_model_path = 'best_resnet_model_imagenet2012v2_classes.keras' # หรือ .h5 หรือ tf format
if os.path.exists(best_model_path):
    best_model = load_model(best_model_path)

    # ไม่จำเป็นต้อง compile ใหม่ถ้า .keras file บันทึกสถานะ optimizer ไว้แล้ว
    # แต่ถ้าต้องการความแน่นอน หรือมีการเปลี่ยน custom objects/metrics ก็สามารถ compile ใหม่ได้
    best_model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR), # ใช้ LR ที่เหมาะสม
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

    val_loss, val_accuracy = best_model.evaluate(val_pipeline)
    print(f"Validation Loss (Best Model): {val_loss}")
    print(f"Validation Accuracy (Best Model): {val_accuracy}")
else:
    print(f"Error: Best model file '{best_model_path}' not found. Training might not have completed or saved a model.")